In [2]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Putting data together**

In [3]:
# prompt: import a .csv dataset

import pandas as pd
mvp_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DL Fall 2024/MSPR predictions/datasets/mvp_train(all terms and course count).csv')
print(mvp_df.head())


   ATTENDANCE  LOW PARTICIPATION  LATE/MISSING ASSIGNMENTS  \
0           0                  0                         2   
1           0                  0                         0   
2           1                  1                         1   
3           0                  0                         0   
4           1                  0                         1   

   OTHER ASSIGNMENTS CONCERNS  LOW TEST SCORES  DANGER of UNSATING  \
0                           0                0                   1   
1                           0                0                   0   
2                           2                0                   0   
3                           0                0                   0   
4                           0                0                   0   

   Course_Count  DESIGNATION_BINARY  
0             3                   1  
1             1                   1  
2             4                   1  
3             1                   1  
4             2 

In [4]:
mvp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4530 entries, 0 to 4529
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   ATTENDANCE                  4530 non-null   int64
 1   LOW PARTICIPATION           4530 non-null   int64
 2   LATE/MISSING ASSIGNMENTS    4530 non-null   int64
 3   OTHER ASSIGNMENTS CONCERNS  4530 non-null   int64
 4   LOW TEST SCORES             4530 non-null   int64
 5   DANGER of UNSATING          4530 non-null   int64
 6   Course_Count                4530 non-null   int64
 7   DESIGNATION_BINARY          4530 non-null   int64
dtypes: int64(8)
memory usage: 283.2 KB


In [5]:
# prompt: show distributions for fields in the mvp_df, show distributions in terms of count and percentage

# Calculate counts and percentages for each column
for col in mvp_df.columns:
  print(f"Distribution for column: {col}")
  counts = mvp_df[col].value_counts()
  percentages = (counts / len(mvp_df)) * 100
  distribution_df = pd.DataFrame({'Count': counts, 'Percentage': percentages})
  print(distribution_df)
  print("-" * 20)

Distribution for column: ATTENDANCE
            Count  Percentage
ATTENDANCE                   
0            3913   86.379691
1             478   10.551876
2             102    2.251656
3              33    0.728477
4               3    0.066225
5               1    0.022075
--------------------
Distribution for column: LOW PARTICIPATION
                   Count  Percentage
LOW PARTICIPATION                   
0                   3881   85.673289
1                    563   12.428256
2                     77    1.699779
3                      9    0.198675
--------------------
Distribution for column: LATE/MISSING ASSIGNMENTS
                          Count  Percentage
LATE/MISSING ASSIGNMENTS                   
0                          3331   73.532009
1                           886   19.558499
2                           235    5.187638
3                            72    1.589404
4                             6    0.132450
--------------------
Distribution for column: OTHER ASSIGNM

**Stratified sampling for train, val, and test data**

In [6]:
# prompt: Split mvp_df into train, validation, and test. Use stratified splitting to maintain class balance across sets.

from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(mvp_df, test_size=0.3, stratify=mvp_df['DESIGNATION_BINARY'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['DESIGNATION_BINARY'], random_state=42)

print("Train set shape:", train_df.shape)
print("Validation set shape:", val_df.shape)
print("Test set shape:", test_df.shape)

Train set shape: (3171, 8)
Validation set shape: (679, 8)
Test set shape: (680, 8)


In [7]:
# prompt: Show distributions of DESIGNATION_BINARY in train_df, val_df, and test_df

def show_distribution(df, name):
    print(f"Distribution for {name}:")
    counts = df['DESIGNATION_BINARY'].value_counts()
    percentages = (counts / len(df)) * 100
    distribution_df = pd.DataFrame({'Count': counts, 'Percentage': percentages})
    print(distribution_df)
    print("-" * 20)

show_distribution(train_df, "train_df")
show_distribution(val_df, "val_df")
show_distribution(test_df, "test_df")

Distribution for train_df:
                    Count  Percentage
DESIGNATION_BINARY                   
1                    2879   90.791548
0                     292    9.208452
--------------------
Distribution for val_df:
                    Count  Percentage
DESIGNATION_BINARY                   
1                     617   90.868925
0                      62    9.131075
--------------------
Distribution for test_df:
                    Count  Percentage
DESIGNATION_BINARY                   
1                     617   90.735294
0                      63    9.264706
--------------------


In [8]:
# prompt: Split train_df into X_train and y_train. Split val_df into X_val and y_val. Split test_df into X_test and y_test.

X_train = train_df.drop('DESIGNATION_BINARY', axis=1)
y_train = train_df['DESIGNATION_BINARY']

X_val = val_df.drop('DESIGNATION_BINARY', axis=1)
y_val = val_df['DESIGNATION_BINARY']

X_test = test_df.drop('DESIGNATION_BINARY', axis=1)
y_test = test_df['DESIGNATION_BINARY']

**SMOTE (balance classes for label)**

In [9]:
# prompt: Use SMOTE on X_train and y_train, to fix unbalance in DESIGNATION_BINARY class distribution

#!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

print()
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

print()
print("Shape of X_train_resampled:", X_train_resampled.shape)
print("Shape of y_train_resampled:", y_train_resampled.shape)

print()
print("Distribution of y_train_resampled:")
print(y_train_resampled.value_counts())


Shape of X_train: (3171, 7)
Shape of y_train: (3171,)

Shape of X_train_resampled: (5758, 7)
Shape of y_train_resampled: (5758,)

Distribution of y_train_resampled:
DESIGNATION_BINARY
1    2879
0    2879
Name: count, dtype: int64


**Baseline model: Logistic**

In [10]:
# prompt: Fit a Logistic regression on train data sets, validate on the validation data sets. Record performance metrics for further comparison.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Initialize and train the Logistic Regression model
logreg_model = LogisticRegression(random_state=42)
logreg_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the validation set
y_val_pred = logreg_model.predict(X_val)

# Evaluate the model
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred)


print("Logistic Regression Performance Metrics (Validation Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

performance_metrics = {
    'model': 'Logistic Regression',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc
}

Logistic Regression Performance Metrics (Validation Set):
Accuracy: 0.8203
Precision: 0.9750
Recall: 0.8233
F1-score: 0.8928
ROC AUC: 0.8068


**Logistic Model: Tuning**

In [10]:
# prompt: Use grid search and cross validation for hyperparameter tuning for the Logistic regression. Record performance metrics.

from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Logistic Regression
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Inverse of regularization strength
    'penalty': ['l1', 'l2'],  # Regularization type
    'solver': ['liblinear', 'saga'] # Solvers that support both penalties
}

# Initialize GridSearchCV
grid_search = GridSearchCV(LogisticRegression(random_state=42, max_iter=1000), param_grid, cv=5, scoring='f1', n_jobs=-1)

# Fit the GridSearchCV to the training data
grid_search.fit(X_train_resampled, y_train_resampled)


# Get the best hyperparameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


print("Best Hyperparameters:", best_params)

# Evaluate the best model on the validation set
y_val_pred = best_model.predict(X_val)

# Calculate performance metrics
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred)


print("Logistic Regression Performance Metrics After Tuning (Validation Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

performance_metrics_log_val = {
    'model': 'Tuned Logistic Regression',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc
}

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_pred)


print("Logistic Regression Performance Metrics After Tuning (Testing set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

performance_metrics = {
    'model': 'Tuned Logistic Regression',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc
}

Best Hyperparameters: {'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}
Logistic Regression Performance Metrics After Tuning (Validation Set):
Accuracy: 0.8424
Precision: 0.9775
Recall: 0.8460
F1-score: 0.9070
ROC AUC: 0.8262
Logistic Regression Performance Metrics After Tuning (Testing set):
Accuracy: 0.8279
Precision: 0.9630
Recall: 0.8428
F1-score: 0.8989
ROC AUC: 0.7627


**Neural Network for binary classification**

In [12]:
# prompt: Fit a Neural Network that would be well suited for binary classification on a relatively small dataset. Fit on train data, evaluate on the validation data. Record performance metrics.

import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the model
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(X_train_resampled.shape[1],)),
    keras.layers.Dropout(0.3),  # Add dropout for regularization
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train_resampled, y_train_resampled, epochs=20, batch_size=32, validation_data=(X_val, y_val))


# Make predictions on the validation set
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int) # Convert probabilities to binary predictions

# Evaluate the model (validation set)
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred_prob)


print("Neural Network Performance Metrics (Validation Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

performance_metrics = {
    'model': 'Neural Network',
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1_score': f1,
    'roc_auc': roc_auc
}

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.5585 - loss: 0.7233 - val_accuracy: 0.7378 - val_loss: 0.6357
Epoch 2/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.6972 - loss: 0.5999 - val_accuracy: 0.7761 - val_loss: 0.5293
Epoch 3/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7177 - loss: 0.5678 - val_accuracy: 0.7555 - val_loss: 0.5151
Epoch 4/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7341 - loss: 0.5529 - val_accuracy: 0.7570 - val_loss: 0.5153
Epoch 5/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7219 - loss: 0.5575 - val_accuracy: 0.7585 - val_loss: 0.4804
Epoch 6/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7309 - loss: 0.5395 - val_accuracy: 0.7658 - val_loss: 0.4953
Epoch 7/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7364 - loss: 0.5522 - val_accuracy: 0.7570 - val_loss: 0.4997
Epoch 8/20
180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7427 - loss: 0.5459 - val_accuracy: 0.

In [13]:
!pip install scikeras

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras

# Define the parameter grid for the neural network
param_grid = {
    'model__units1': [16, 32, 64, 128],
    'model__units2': [8, 16, 32, 64],
    'model__dropout': [0.2, 0.3, 0.4],
    'batch_size': [32, 64],
    'epochs': [30]
}

# Function to create and compile the model
def create_model(units1, units2, dropout, optimizer='adam'):
    model = keras.Sequential([
        keras.layers.Dense(units1, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(units2, activation='relu'),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Use KerasClassifier for Scikit-learn compatibility
model = KerasClassifier(model=create_model, verbose=0)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_result = grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and score
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.7537732189563284 using {'batch_size': 64, 'epochs': 30, 'model__dropout': 0.2, 'model__units1': 128, 'model__units2': 64}


In [15]:
!pip install scikeras

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras

#best model from tuning above
model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the best model on the validation set
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

#Evaluate the model
#Ensure the labels are binary
y_val = LabelEncoder().fit_transform(y_val)

# Compute metrics (val)
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred_prob)

# Print metrics
print("Tuned Neural Network Performance Metrics (Validation Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Evaluate the best model on the validation set
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

#Evaluate the model
#Ensure the labels are binary
y_test = LabelEncoder().fit_transform(y_test)

# Compute metrics (test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_pred)

# Print metrics
print("Tuned Neural Network Performance Metrics (Test Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6881 - loss: 0.5826 - val_accuracy: 0.7143 - val_loss: 0.5105
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7394 - loss: 0.5442 - val_accuracy: 0.7099 - val_loss: 0.5837
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7404 - loss: 0.5532 - val_accuracy: 0.7820 - val_loss: 0.5001
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7426 - loss: 0.5306 - val_accuracy: 0.7216 - val_loss: 0.5159
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7412 - loss: 0.5365 - val_accuracy: 0.7349 - val_loss: 0.4747
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7506 - loss: 0.5257 - val_accuracy: 0.7452 - val_loss: 0.5161
Epoch 7/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7565 - loss: 0.5215 - val_accuracy: 0.7334 - val_loss: 0.5249
Epoch 8/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7564 - loss: 0.5239 - val_accuracy: 0.74

In [16]:
#trying with 3 hidden layers

!pip install scikeras

from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras

# Define the parameter grid for the neural network
param_grid = {
    'model__units1': [16, 32, 64, 128],
    'model__units2': [8, 16, 32, 64],
    'model__units3': [4, 8, 16, 32],
    'model__dropout': [0.2, 0.3],
    'batch_size': [64],
    'epochs': [30]
}

# Function to create and compile the model
def create_model(units1, units2, units3, dropout, optimizer='adam'):
    model = keras.Sequential([
        keras.layers.Dense(units1, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(units2, activation='relu'),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(units3, activation='relu'),
        keras.layers.Dropout(dropout),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Use KerasClassifier for Scikit-learn compatibility
model = KerasClassifier(model=create_model, verbose=0)

# Use GridSearchCV to find the best hyperparameters
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='f1', n_jobs=-1)
grid_result = grid_search.fit(X_train_resampled, y_train_resampled)

# Print the best parameters and score
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.7572121413599442 using {'batch_size': 64, 'epochs': 30, 'model__dropout': 0.2, 'model__units1': 128, 'model__units2': 64, 'model__units3': 4}


In [11]:
!pip install scikeras

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow import keras

#best model from tuning above (execution failed to some code after the tuning)
model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train_resampled.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(4, activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(1, activation='sigmoid')
    ])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
              loss='binary_crossentropy',metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluate the best model on the validation set
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype(int)

#Evaluate the model
#Ensure the labels are binary
y_val = LabelEncoder().fit_transform(y_val)

# Compute metrics (val)
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)
roc_auc = roc_auc_score(y_val, y_val_pred_prob)

# Print metrics
print("Tuned Neural Network Performance Metrics 3 hidden (Validation Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

# Evaluate the best model on the validation set
y_test_pred_prob = model.predict(X_test)
y_test_pred = (y_test_pred_prob > 0.5).astype(int)

#Evaluate the model
#Ensure the labels are binary
y_test = LabelEncoder().fit_transform(y_test)

# Compute metrics (test)
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)
roc_auc = roc_auc_score(y_test, y_test_pred)

# Print metrics
print("Tuned Neural Network Performance Metrics 3 hidden (Test Set):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6602 - loss: 0.6010 - val_accuracy: 0.7482 - val_loss: 0.5309
Epoch 2/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7182 - loss: 0.5661 - val_accuracy: 0.7541 - val_loss: 0.5418
Epoch 3/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7382 - loss: 0.5613 - val_accuracy: 0.7658 - val_loss: 0.4813
Epoch 4/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7287 - loss: 0.5459 - val_accuracy: 0.7069 - val_loss: 0.5408
Epoch 5/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7384 - loss: 0.5508 - val_accuracy: 0.7305 - val_loss: 0.4841
Epoch 6/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7296 - loss: 0.5551 - val_accuracy: 0.7275 - val_loss: 0.5440
Epoch 7/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7251 - loss: 0.5584 - val_accuracy: 0.8306 - val_loss: 0.4693
Epoch 8/100
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7335 - loss: 0.5495 - val_accuracy: 0.7717 - val_loss: 0.4